# Quantinuum Simulators, Emulators and Quantum Hardware, via Nexus


This notebook outlines how to target and configure Quantinuum hardware or emulators.

[Go to Full Example Quantinuum Systems Workflow using qnexus](#example-quantinuum-systems-workflow)

For more extensive documentation on emulators and hardware, please see the [Quantinuum Systems documentation](https://docs.quantinuum.com/quantinuum-systems)

In [ ]:
import qnexus as qnx
from qnexus.models.language import Language

## Available Quantinuum Machines

You can view which Quantinuum machines you have access to with the `devices` module.


In [ ]:
qnx.devices.get_all(issuers=[qnx.devices.IssuerEnum.QUANTINUUM]).df()

You can select which Quantinuum machines you'd like to target via the `device_name` parameter in the `QuantinuumConfig` you pass as part of a job submission.

In [ ]:
# Create a configuration to target the H1-1LE noiseless simulator
my_quantinuum_config = qnx.QuantinuumConfig(
    device_name="H1-1LE",
)

If you are targeting a hardware-hosted Quantinuum device (including hardware-hosted emulators), you can check whether the device is online or offline.

N.B. this will not work for cloud-hosted Quantinuum emulators which are generally always online.

In [ ]:
my_quantinuum_hw_config = qnx.QuantinuumConfig(
    device_name="H1-1",
)

qnx.devices.status(my_quantinuum_hw_config)

## Calculating Cost

Nexus provides the ability to find out the cost of running circuit executions on Quantinuum quantum computers (e.g. `H1-1`) and Quantinuum-hosted emulators (e.g. `H1-1E`). 

NB this is different from the cloud-hosted simulators (e.g. `H1-1LE`) or emulators (e.g. `H1-Emulator`) which are accounted for in your Simulation quota (measured in seconds), for which we do not offer cost estimation.

The `syntax_checker` argument should be the corresponding syntax checker device (e.g. "H1-1SC" for "H1-1E" or "H1-1").

```python
qnx.circuits.cost(
    circuit_ref=your_circuit_ref,
    n_shots=1000,
    backend_config=qnx.QuantinuumConfig(device_name="H1-1"),
    syntax_checker="H1-1SC",
)

# -> returns a float representing the cost in HQC
```




## Emulator Error Configuration

By default, both the cloud-hosted (e.g. `H1-Emulator`) and Quantinuum-hosted (e.g. `H1-1E`) emulators will model Quantinuum hardware. If desired this noise can be configured.

For documentation on noise configuration, please see the [Quantinuum emulator noise model documentation](https://docs.quantinuum.com/systems/user_guide/emulator_user_guide/noise_model.html)

In [ ]:
from quantinuum_schemas.models.quantinuum_systems_noise import UserErrorParams

my_params = UserErrorParams(
    p1=0.1,
    p2=0.5,
    # adjust as needed according to noise model documentation
)

backend_config = qnx.QuantinuumConfig(
    device_name="H1-Emulator",  # Or another emulator such as H1-1E
    error_params=my_params,
)

In [ ]:
# Alternatively you can pass in the noise model as a dictionary
backend_config = qnx.QuantinuumConfig(
    device_name="H1-Emulator", error_params={"p1": 0.1, "p2": 0.5}
)

## Compiler Options


By default, additional compliation will NOT occur in the Quantinuum submission stack (e.g. for quantum hardware or hardware-hosted emulators like `H1-1E`). However if desired, additional compilition can be configured.

Certain compiler flags can be passed on to the compiler in the Quantinuum submission stack. This is distinct from the TKET-based compilation that happens when you compile circuits in Nexus.

You can configure compiler options in the `QuantinuumConfig` by passing `no_opt=False` and `QuantinuumCompilerOptions` as arguments.

If you would like more information on the compilation configuration available, please get in touch with Quantinuum suppport.

In [ ]:
from quantinuum_schemas.models.backend_config import QuantinuumCompilerOptions

# Allow the hardware compiler to decide what 2-qubit gate to rebase to.
compiler_flags = {"noreduce": True}

backend_config = qnx.QuantinuumConfig(
    device_name="H1-1",
    no_opt=False,
    compiler_options=QuantinuumCompilerOptions(**compiler_flags),
)

## Submission Language selection

You can specify the submission language for execute job submissions to Quantinuum Systems, available options include `QIR`, `AUTO` and `OPENQASM 2.0`.

```python
execute_job_ref = qnx.start_execute_job(
    circuits=my_circuit_ref,
    name=f"My Execute Job from {datetime.now()}",
    n_shots=[100],
    backend_config=backend_config,
    project=my_project_ref,
    language="QIR",
)
```

## Queueing for Quantinuum Systems hardware and emulators

Quantinuum Nexus is the default platform for submitting to Quantinuum hardware and emulators, however these have their own dedicated job queue that is 'below' Nexus.

When you submit a job to Quantinuum hardware or emulators, they will be entered into a fair-share queuing system. When the selected device is available the fair-share queueing algorithm will select a job approximately as follows:

- Select jobs queued for active batches.
- Select an Organisation that has used the device the least in recent history.
    - Select a job from that Organisation that was submitted under a User Group with the highest Group priority (1 being highest, 5 being default, 10 being lowest).
    - Select a job from that Organisation that was submitted by a User with the highest User priority (same ordering as above).


Your organization admin can help assign the appropriate priority to you or a User Group you are a member of.


## Batching

If you expect to run multiple circuits, or several circuits in succession (such as during a variational project) you can request that the Quantinuum submission queue 'batches' your circuits together so they avoid queueing seperately. Circuits submitted to an existing batch will run immediately, provided they are submitted within 60 seconds of the completion of the last circuit you submitted to the batch. 


If desired Nexus will attempt to automatically handle the batching of circuits when submitted in a list (such as an execute job with multiple circuits) or in succession (multiple execute job submissions).

You can request that Nexus attempts to batch your circuits by setting the `attempt_batching` flag in the `QuantinuumConfig`:

In [ ]:
backend_config = qnx.QuantinuumConfig(device_name="H1-1E", attempt_batching=True)

# Example Quantinuum workflow

The following is a minimal example workflow targetting the Quantinuum emulator `H1-1E` to compile and execute a circuit with a custom noise model configuration.

In [ ]:
from datetime import datetime

from pytket import Circuit
from quantinuum_schemas.models.quantinuum_systems_noise import UserErrorParams

import qnexus as qnx

In [ ]:
# View Quantinuum devices that are available to the logged in user
qnx.devices.get_all(issuers=["QUANTINUUM"]).df()

In [ ]:
my_project_ref = qnx.projects.get_or_create(name="My H1-1E Project")

In [ ]:
# Specify our config to target the H1-1E device, with a custom noise model

my_params = UserErrorParams(
    # Linearly scale the probability of dephasing causing a fault.
    memory_scale=0.5,
)

backend_config = qnx.QuantinuumConfig(device_name="H1-1E", error_params=my_params)

In [ ]:
circuit = Circuit(2).H(0).CX(0, 1).measure_all()

my_circuit_ref = qnx.circuits.upload(
    name=f"My Q Systems Circuit from {datetime.now()}",
    circuit=circuit,
    project=my_project_ref,
)

In [ ]:
# Compile the circuit on Nexus using our hosted TKET compilation service

compile_job = qnx.start_compile_job(
    circuits=[my_circuit_ref],
    name=f"My Compile Job from {datetime.now()}",
    optimisation_level=1,
    backend_config=backend_config,
    project=my_project_ref,
)

qnx.jobs.wait_for(compile_job)

compiled_circuits = [item.get_output() for item in qnx.jobs.results(compile_job)]

In [ ]:
# Double check the HQC cost of running the circuit
execution_cost = qnx.circuits.cost(
    circuit_ref=compiled_circuits[0],
    n_shots=1000,
    backend_config=qnx.QuantinuumConfig(device_name="H1-1E"),
    syntax_checker="H1-1SC",
)

print(execution_cost)

In [ ]:
# Submit the job for execution on the H1-Emulator

execute_job_ref = qnx.start_execute_job(
    circuits=compiled_circuits,
    name=f"My Execute Job from {datetime.now()}",
    n_shots=[100] * len(compiled_circuits),
    backend_config=backend_config,
    project=my_project_ref,
    language=Language.QIR,
)

execute_job_ref.df()

In [ ]:
qnx.jobs.wait_for(execute_job_ref)

In [ ]:
# Retrieve a ExecutionResultRef for every Circuit that was executed
execute_job_result_refs = qnx.jobs.results(execute_job_ref)

# Get a pytket BackendResult for the execution
result = execute_job_result_refs[0].download_result()

result.get_counts()